# AMS516 Homework 1. Continuous-time stochastic linear quadratic regulators (LQR)

## Juan Perez Osorio

# **1. An infinite-horizon discounted stochastic LQR problem**

Consider the following continuous-time dynamics:

$$dx_t = (Ax_t + Bu_t)dt + Gdw_t$$

where:

* $x_t \in \mathbb{R}^n$ is the state vector
* $u_t \in \mathbb{R}^m$ is the control input
* $w_t$ is a standard $r$-dimensional Wiener process
* $G \in \mathbb{R}^{n \times r}$ is the noise gain matrix

The discounted quadratic cost is given by:

$$J(x_0; u) = \mathbb{E}\left[ \int_0^\infty e^{-\rho t}(x_t^\intercal Q x_t + u_t^\intercal R u_t) dt \right]$$

with:

* Discount rate $\rho > 0$
* $Q \geq 0$ (positive semidefinite)
* $R \geq 0$ (positive semidefinite)

The goal is to find optimal feedback control $u_t = -Kx_t$ minimizing $J$.

The Hamilton-Jacobi-Bellman (HJB) equation for the discounted cost is:

$$\rho V(x) = \min_u \left\{x^\intercal Q x + u^\intercal R u + \nabla V(x)^\intercal (Ax + Bu) + \frac{1}{2} \text{tr}(GG^\intercal \nabla^2 V) \right\}$$

The value function $V(x)$ and the optimal control $u$ have the form:

$$V(x) = x^\intercal P x + c$$
$$u = -R^{-1} B^\intercal P x$$

Moreover, $P$ satisfies the discounted continuous algebraic Riccati equation (CARE)

$$A^\intercal P + PA - P B R^{-1} B^\intercal P + Q - \rho P = 0$$

and $c$ is given by:

$$c = \frac{\operatorname{tr}(G G^T P)}{\rho}.$$

Consider the problem with the following parameters:

$$
\rho = 0.1, \quad A = \begin{bmatrix} 0 & 1 \\ -2 & -3 \end{bmatrix}, \quad B = \begin{bmatrix} 0 \\ 1 \end{bmatrix}, \quad G = \begin{bmatrix} 0.1 & 0 \\ 0 & 0.1 \end{bmatrix}, \quad Q = \begin{bmatrix} 1 & 0 \\ 0 & 1 \end{bmatrix}, \quad R = [1].
$$

(a). Use the analytical result above to compute the value function $V(x)$ and optimal control $u$ (see the sample code in Appendix 1)

In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
from scipy.linalg import solve_continuous_are, solve_lyapunov, eigvals

In [10]:
rho: float = 0.1

A = np.array([[0, 1], [-2, -3]])
B = np.array([[0], [1]])
G = np.array([[0.1, 0], [0, 0.1]])
Q = np.array([[1, 0], [0, 1]])
R = np.array([[1]])

We can see that when checking the library for the function ``solve_continuos_are``, we found that it solves the continuos-time algebraic Riccati equation (CARE):

$$
\begin{equation}
A^H X + XA - X B R^{-1} B^H X + Q = 0
\end{equation}
$$

But we now that $P = X$ satisfies the discounted continuous algebraic Riccati equation (CARE):

$$
\begin{equation*}
A^\intercal P + PA - P B R^{-1} B^\intercal P + Q - \rho P = 0
\end{equation*}
$$

So we need to turn our last equation into the form of (1) so we can apply the scipy method. To do so we manipulate the expression in the following way

$$
\begin{split}
A^\intercal P + PA - P B R^{-1} B^\intercal P + Q - \rho P &= 0 \\
A^\intercal P + PA - P B R^{-1} B^\intercal P + Q  &= \rho P\\
A^\intercal P + PA - P B R^{-1} B^\intercal P + Q  &= \frac{1}{2}\rho (IP + PI)\\
\left(A^\intercal P - \frac{1}{2} \rho IP \right) + \left(PA - \frac{1}{2} \rho PI \right) - P B R^{-1} B^\intercal P + Q  &= 0 \\
\left(A^\intercal - \frac{1}{2} \rho I \right)P + P\left(A - \frac{1}{2} \rho I \right) - PBR^{-1} B^\intercal P + Q  &= 0 \\
\end{split}
$$

Letting $A_{\text{disc}} = A - \frac{1}{2} \rho I$, we can see that the previous equation reduces to 

$$
A^{\intercal}_{\text{disc}}P + PA_{\text{disc}} - PBR^{-1} B^\intercal P + Q  = 0
$$

we can see that our final expression is now in the same form as (1) and hence, we can apply the scipy method

In [ ]:
I = np.identity(A.shape[0])

# Modify A so that we can use the solve_continuos_are method from scipy
A_discounted = A - (1/2) * rho * I

P = solve_continuous_are(a = A_discounted, b = B, q = Q, r = R)

The optimal gain matrix $K$ is defined to be

$$K = R^{-1} B^\intercal P$$

so we have that 

$$u = -Kx$$

In [21]:
# Optimal gain matrix
K = np.linalg.inv(R) @ B.T @ P

Replacing this in the time dynamics we have 

$$
\begin{split}
dx_t &= (Ax_t + Bu_t)dt + Gdw_t \\
     &= (Ax_t - BK x_t)dt + Gdw_t \\
     &= (A- BK)x_tdt + Gdw_t \\
     &= (A_{\text{closed}})x_tdt + Gdw_t
\end{split}
$$

Where we define $A_{\text{closed}}$ to be the closed loop matrix and it describes the dynamics of a system with feedback control law, i.e., the closed-loop system is obtained by substituting the optimal control $u$ into the dynamics of the system.

A nice way to checking the stability of the system is by checking the eigenvalues of the closed loop matrix

> we can summarize how the two-dimensional linear dynamical system's stability depends on  $\operatorname{Tr}(A)$ and  $det(A)$ in a simple diagram as shown

![stability](./images/stability.png)

Hence we have to check that all the eigenvalues of the closed loop matrix are negative so that our system is stable; we do this procedure to confirm that the solution $P$ from the (discounted) CARE produces a stabilizing controller $K$. If any eigenvalue had a positive real part, the controller would be useless as the state would blow up.

In [24]:
# Stability analysis
A_cl = A - B @ K
eig_A_cl = eigvals(A_cl)

print("\nEigenvalues of A_cl:")
print(eig_A_cl)

# Check if all real parts are negative (system is stable)
is_stable = np.all(np.real(eig_A_cl) < 0)
print(f"Is the closed-loop system stable? {is_stable}")


Eigenvalues of A_cl:
[-0.98838969+0.j -2.23620796+0.j]
Is the closed-loop system stable? True


Let’s define $X_t = \mathbb{E}[x_t x_t^T]$. This is the covariance matrix of the state at time $t$. Its evolution is governed by the SDE. Itô's Lemma tells us how to find the differential $d(x_t x_t^T)$.

$$
\begin{split}
d(x_t x_t^\intercal) =& \left[d(x_t)\right]x^{\intercal}_{t} + x_t \left[d(x^{\intercal}_{t})\right] + \left[d(x_t)\right]\left[d(x^{\intercal}_{t})\right] \\

=& \left[A_{\text{closed}}x_tdt + Gdw_t\right]x^{\intercal}_{t} + x_t \left[(A_{\text{closed}}x_tdt + Gdw_t)^\intercal\right] + \left[d(x_t)\right]\left[d(x^{\intercal}_{t})\right] \\

\end{split}
$$

but we know that 

$$[d(x_t)][d(x_t)]^\intercal = (G dw_t)(G dw_t)^\intercal = G (dw_t dw_t^\intercal) G^\intercal = G (I dt) G^\intercal = G G^\intercal dt$$

Hence, we have 

$$ d(x_t x_t^\intercal) = \left(A_{\text{closed}}x_tdt + Gdw_t\right)x^{\intercal}_{t} + x_t \left(A_{\text{closed}}x_tdt + Gdw_t\right)^\intercal + G G^\intercal dt$$

We want an equation for $dX_t = d(\mathbb{E}[x_t x_t^\intercal]) = \mathbb{E}[d(x_t x_t^\intercal)]$.

* $\mathbb{E}[G dw_t x_t^\intercal] = 0$ and $\mathbb{E}[x_t dw_t^\intercal G^\intercal] = 0$ because the current state $x_t$ is uncorrelated with the future noise $dw_t$.

* $\mathbb{E}[A_{cl} x_t x_t^\intercal dt] = A_{cl} \mathbb{E}[x_t x_t^\intercal] dt = A_{cl} X_t dt$

* $\mathbb{E}[x_t x_t^\intercal A_{cl}^\intercal dt] = X_t A_{cl}^\intercal dt$

* $\mathbb{E}[G G^\intercal dt] = G G^\intercal dt$

Hence we have that

$$ dX_t = \mathbb{E}[d(x_t x_t^\intercal)] = A_{\text{closed}}X_tdt + X_{t}A_{\text{closed}}^{\intercal}dt + GG^\intercal dt$$

$$ \frac{dX_t}{dt} = A_{\text{closed}}X_t + X_{t}A_{\text{closed}}^{\intercal} + GG^\intercal$$

At steady-state, the system's statistical properties don't change anymore. This means the covariance matrix $X_t$ settles to a constant matrix $\Sigma$, so its derivative becomes zero $dX_t / dt = 0$. Hence, at steady state we have that

$$
A_{\text{closed}}\Sigma + \Sigma A_{\text{closed}}^{\intercal} + GG^\intercal = 0
$$ 

This is the famous **Lyapunov equation**. To solve it we use the ``solve_lyapunov`` from the scipy library

In [25]:
# Steady state covariance
GGt = G @ G.T

# Solves the continuous Lyapunov equation AX + XA^H = Q
S = solve_lyapunov(A_cl, -GGt)

# The discounted value constants are given by
c = np.trace(GGt @ P) / rho

# Example: initial state and optimal discounted cost
x0 = np.array([1.0, 0])
J0 = x0.T @ P @ x0 + c

print('rho = ', rho)
print('P = \n', P)
print('K = \n', K)
print('Sigma = \n', S)
print('Value constant c = ', c)
print('J(x0) = ', J0)

rho =  0.1
P = 
 [[1.14817499 0.2102449 ]
 [0.2102449  0.22459765]]
K = 
 [[0.2102449  0.22459765]]
Sigma = 
 [[ 0.00954679 -0.005     ]
 [-0.005       0.00497774]]
Value constant c =  0.13727726484530767
J(x0) =  1.2854522598310418


(b) Show that the optimal control $u$ in (3) is given by $u^∗ = −R^{−1}B^\intercal \nabla V$ . Plug $u^∗$ into (3), we obtain a differential equation of $V$. Use the PINN method to solve the obtained
equation for $V$ and $u$. Denote your solution by $\hat{V}$ and $\hat{u}$.

We know that the HJB equation for the discounted cost is 

$$\rho V(x) = \min_u \left\{x^\intercal Q x + u^\intercal R u + \nabla V(x)^\intercal (Ax + Bu) + \frac{1}{2} \text{tr}(GG^\intercal \nabla^2 V) \right\}$$

as we can see, not all terms inside the minimization objective contain $u$, hence, we can split this in two parts

$$\rho V(x) = \min_u \left\{u^\intercal R u + \nabla V(x)^\intercal Bu \right\} + \left[x^\intercal Q x + \nabla V(x)^\intercal Ax + \frac{1}{2} \text{tr}(GG^\intercal \nabla^2 V) \right]$$

Since we want to find the $u$ that minimizes the expression, we can focus solely on the part that depends on $u$, let's define this part by $f$

$$ f(u) = u^\intercal R u + \nabla V(x)^\intercal Bu $$

To find the minimum of the function $g(u)$ with respect to the vector $u$, we take its gradient (derivative with respect to $u$) and set it equal to zero. Computing the Gradient $\nabla_u f(u)$ we obtain that 

$$\nabla_u f(u) = (R + R^\intercal)u + B^\intercal \nabla V(x)$$

But since $R$ is symmetric we have that

$$\nabla_u f(u) = 2Ru + B^\intercal \nabla V(x) = 0$$
$$2Ru = -B^\intercal \nabla V(x)$$
$$u^* = -\frac{1}{2}R^{-1}B^\intercal \nabla V(x)$$

Plugging this into the HJB equation for the discounted cost we can notice that

$$\nabla V(x)^\intercal B u^* = \nabla V(x)^T B (-R^{-1}B^T \nabla V) = -\frac{1}{2}\nabla V^\intercal B R^{-1} B^\intercal \nabla V$$

$$
\begin{split}
{u^*}^\intercal R u^* &= \left(-\frac{1}{2}R^{-1}B^\intercal \nabla V\right)^\intercal R \left(-\frac{1}{2}R^{-1}B^\intercal \nabla V\right) \\
                      &= \frac{1}{4}\nabla V^\intercal B (R^{-1})^\intercal RR^{-1}B^\intercal \nabla V \\
                      &= \frac{1}{4}\nabla V^\intercal B R^{-1} B^\intercal \nabla V \\
\end{split}
$$ 

Now we plug these back in. Notice that 

$$(u^*)^\intercal R u^* + \nabla V(x)^\intercal B u^* = \frac{1}{4}\nabla V^\intercal B R^{-1} B^\intercal \nabla V - \frac{1}{2}\nabla V^\intercal B R^{-1} B^\intercal \nabla V = - \frac{1}{4}\nabla V^\intercal B R^{-1} B^\intercal \nabla V$$

Therefore we have that the HJB equation for the discounted cost takes the form

$$\rho V(x) = x^\intercal Q x + \nabla V(x)^\intercal Ax - \frac{1}{4}\nabla V^\intercal B R^{-1} B^\intercal \nabla V + \frac{1}{2} \text{tr}(GG^\intercal \nabla^2 V)$$

So, we can define our loss function to be

$$L_{\operatorname{physics}} = \left| \rho V(x) - x^\intercal Q x - \nabla V(x)^\intercal Ax + \frac{1}{4}\nabla V(x)^\intercal B R^{-1} B^\intercal \nabla V(x) - \frac{1}{2} \text{tr}(GG^\intercal \nabla^2 V(x))\right|$$

Now we're going to set up the PINN in torch

1. Import required libraries

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim

In [12]:
# Set device: allows us to specify whether computations should occur on the CPU or GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Test to see where the tensor was created
tensor_on_device = torch.rand(5, device=device)
print(f"Tensor created directly on {tensor_on_device.device}")

# Convert the system parameters to torch tensors
A = torch.tensor([[0.0, 1.0], [-2.0, -3.0]], device = device)
B = torch.tensor([[0.0], [1.0]], device = device)
G = torch.tensor([[0.1, 0.0], [0.0, 0.1]], device = device)
Q = torch.tensor([[1.0, 0.0], [0.0, 1.0]], device = device)
R = torch.tensor([[1.0]], device = device)
R_inv = torch.linalg.inv(R)

# Pre-compute useful matrices
B_Rinv_BT = B @ R_inv @ B.T
GGt = G @ G.T

Tensor created directly on cpu


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Define the network

model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation = 'tanh', input_shape = (1,)),
    tf.keras.layers.Dense(10, activation = 'tanh'),
    tf.keras.layers.Dense(1)
])

# Define the physics loss and boundary loss
def loss(model, x, f):
    with tf.GradientTape(persistent = True) as tape:
        tape.watch(x)
        u = model(x)
        u_x = tape.gradient(u, x)
        u_xx = tape.gradient(u_x, x)
    
    physics_loss = tf.reduce_mean(tf.square(u_xx + f))

    # Boundary loss
    u_0 = model(tf.convert_to_tensor([[0.0]], dtype = tf.float32))
    u_1 = model(tf.convert_to_tensor([[1.0]], dtype = tf.float32))
    boundary_loss = tf.square(u_0) + tf.square(u_1)

    return physics_loss + boundary_loss

# Training data
x = tf.convert_to_tensor(np.linspace(0, 1, 100).reshape(-1, 1), dtype = tf.float32)

f = np.pi ** 2 * np.sin(np.pi * x)

# Optimizer
optimizer = tf.keras.optimizers.Adam()

for epoch in range(2000):
    with tf.GradientTape() as tape:
        current_loss = loss(model, x, f)
    gradients = tape.gradient(current_loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    if epoch % 100 == 0:
        print(f'Epoch {epoch}: Loss = {current_loss.numpy()}')

# Evaluate the trained model
u_pinn = model.predict(x)
u_exact = np.sin(np.pi * x)

# Plot the result
plt.figure(figsize=(14, 6))
plt.subplot(1, 2, 1)
plt.plot(x, u_pinn, label = "PINN Solution")
plt.plot(x, u_exact, label = "Exact Solution")
plt.legend()

plt.subplot(1, 2, 1)
plt.plot(x, u_pinn - u_exact, '--', label = "Error")
plt.legend()
plt.tight_layout()
plt.show()